In [20]:
from dotenv import load_dotenv
import re
import roman
import json
import os
import redis
from tmdbv3api import TMDb, TV, Season, Episode
from tmdbv3api.as_obj import AsObj
import xml.etree.ElementTree as ET

load_dotenv()

True

In [21]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

Test Key:

In [22]:
# Example: Set and get a key
r.set('key', 'value')

True

In [23]:
print(r.get('key'))  # b'value'

value


In [24]:
# Add the media file metadata from the NFO file
metadata = {
    'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'outline': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'lockdata': 'true',
    'dateadded': '2022-10-19 19:46:52',
    'title': 'Cowboy Bebop',
    'originaltitle': 'カウボーイビバップ',
    'writer': 'Hajime Yatate',
    'credits': 'Hajime Yatate',
    'trailer': 'plugin://plugin.video.youtube/?action=play_video&videoid=RI08P5SaJNU',
    'rating': '8.9',
    'year': '1998',
    'mpaa': 'TV-14',
    'imdb_id': 'tt0213338',
    'tmdbid': '30991',
    'premiered': '1998-04-02',
    'releasedate': '1998-04-02',
    'enddate': '1999-04-23',
    'runtime': '25',
    'genre': 'Western',
    'studio': 'WOWOW Prime',
    'tag': 'space western',
    'tvdbid': '76885',
    'art': '',
    'actor': '',
    'id': '76885',
    'episodeguide': '',
    'season': '-1',
    'episode': '-1',
    'status': 'Ended'
}

# Use hset with mapping to add the metadata
r.hset('media:cowboy_bebop', mapping=metadata)

29

In [25]:
# Retrieve metadata
print(r.hgetall('media:cowboy_bebop'))

{'season': '-1', 'episodeguide': '', 'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.', 'tag': 'space western', 'status': 'Ended', 'art': '', 'title': 'Cowboy Bebop', 'premiered': '1998-04-02', 'lockdata': 'true', 'mpaa': 'TV-14', 'studio': 'WOWOW Prime', 'runtime': '25', 'episode': '-1', 'originaltitle': 'カウボーイビバップ', 'tmdbid': '30991', 'dateadded': '2022-10-19 19:46:52', 'actor': '', 'trailer': 'plugin://plugin.video.youtube/?action=play_video&videoid=RI08P5SaJNU', 'imdb_id': 'tt0213338', 'enddate': '1999-04-23', 'tvdbid': '76885', 'releasedate': '1998-04-02', 'outline': 'In 2071, roughly fifty years after an ac

Script to process nfo

In [26]:
# # Load and parse the NFO file
# tree = ET.parse('tvshow.nfo')
# root = tree.getroot()

# # Extract metadata from the NFO file
# metadata = {}
# for child in root:
#     metadata[child.tag] = child.text

# metadata

Functions

In [72]:
#Geeks 4 geeks edit distance
def editDistance(str1, str2):
    # Get the lengths of the input strings
    m = len(str1)
    n = len(str2)

    # Initialize a list to store the current row
    curr = [0] * (n + 1)

    # Initialize the first row with values from 0 to n
    for j in range(n + 1):
        curr[j] = j

    # Initialize a variable to store the previous value
    previous = 0

    # Loop through the rows of the dynamic programming matrix
    for i in range(1, m + 1):
        # Store the current value at the beginning of the row
        previous = curr[0]
        curr[0] = i

        # Loop through the columns of the dynamic programming matrix
        for j in range(1, n + 1):
            # Store the current value in a temporary variable
            temp = curr[j]

            # Check if the characters at the current positions in str1 and str2 are the same
            if str1[i - 1] == str2[j - 1]:
                curr[j] = previous
            else:
                # Update the current cell with the minimum of the three adjacent cells
                curr[j] = 1 + min(previous, curr[j - 1], curr[j])

            # Update the previous variable with the temporary value
            previous = temp

    # The value in the last cell represents the minimum number of operations
    return curr[n]

def parse_filename(filename):
    # Regular expressions for season and episode
    filename = re.sub(r'\[.*?\]', '', filename)
    filename = re.sub(r'\(.*?\)', '', filename)
    patterns = [
        r's(?P<season>\d{4})e(?P<episode>\d{2})',   # Matches s1900e01 format
        r'S(?P<season>\d{4})E(?P<episode>\d{2})',   # Matches S1900E01 format
        r'S(?P<season>\d{2})E(?P<episode>\d{2})',  # Matches S01E01 format
        r'S(?P<season>\d{2}) E(?P<episode>\d{2})',  # Matches S01 E01 format
        r's(?P<season>\d{2})e(?P<episode>\d{2})',   # Matches s01e01 format
        r'(?P<season>\d{4})x(?P<episode>\d{2})',   # Matches 1900x01 format
        r'(?P<season>\d{2})x(?P<episode>\d{2})',    # Matches 01x01 format
        r'(?P<season>\d{2})\.(?P<episode>\d{2})',   # Matches 01.01 format
        r'(?P<season>\d{1})X(?P<episode>\d{2})',    # Matches 1X01 format with single-digit season
        r'(?P<season>\d)X(?P<episode>\d{2})',       # Matches 1X01 format
        r'(?P<season>\d{1})E(?P<episode>\d{2})',    # Matches 1E01 format
        r'S(?P<season>\d{1}) E(?P<episode>\d{2})',       # Matches S3 E12 format
        r'S(?P<season>\d{1}) E(?P<episode>\d{1})',       # Matches S3 E1 format
    ]
    
    # Check for season and episode numbers
    for pattern in patterns:
        match = re.search(pattern, filename)
        if match:
            season_num = match.group('season')
            episode_num = match.group('episode')

            try:
                season_num = int(season_num)
            except ValueError:
                raise ValueError("Season format is invalid")
            try:
                episode_num = int(episode_num)
            except ValueError:
                raise ValueError("Episode format is invalid")
            return {'season': season_num, 'episode': episode_num}
        
    roman_pattern = r'M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})'
    roman_patterns = [
        fr'(?P<season>{roman_pattern}) - (?P<episode>\d{{2}})'
    ]

    for pattern in roman_patterns:
        match = re.search(pattern, filename)
        if match:
            season_num = match.group('season')
            if season_num:
                try:
                    season_num = roman.fromRoman(season_num.upper())
                except roman.InvalidRomanNumeralError:
                    raise ValueError("Season format is invalid")
            else:
                season_num = 1
            
            episode_num = match.group('episode')
            try:
                episode_num = int(episode_num)
            except ValueError:
                raise ValueError("Episode format is invalid")
            
            return {'season': season_num, 'episode': episode_num}

    # Regular expression for episode only
    episode_patterns = [
        r'E(?P<episode>\d{2})',  # Matches E01 format
        r'e(?P<episode>\d{2})',  # Matches e01 format
        r'(?P<episode>\d{2})'    # Matches 01 format
    ]
    
    # Check for episode numbers only
    for pattern in episode_patterns:
        match = re.search(pattern, filename)
        if match:
            episode_num = match.group('episode')
            try:
                # Attempt to convert season to integer first
                episode_num = int(episode_num)
            except ValueError:
                # If conversion fails, attempt to convert Roman numeral to integer
                try:
                    episode_num = roman.fromRoman(episode_num.upper())
                except roman.InvalidRomanNumeralError:
                    raise ValueError("Episode format is invalid")
            return {'season': 1, 'episode': episode_num}
    
    return {'season': 0, 'episode': "unknown"}

def parse_filenames(filenames):
    parsed_info = []
    for filename in filenames:
        info = parse_filename(filename)
        if info:
            parsed_info.append({'filename': filename, 'season': info.get('season'), 'episode': info.get('episode')})
        else:
            parsed_info.append({'filename': filename, 'season': None, 'episode': None})
    return parsed_info

def clean_tv_show_title(title):
    title = re.sub(r'(x264|x265|HVEC).*', '', title, flags=re.IGNORECASE)
    title = re.sub(r'\[.*?\]', '', title)
    title = re.sub(r'\(.*?\)', '', title)
    title = re.sub(r'\.(avi|mkv|mp4|WEBRip|web|AMZN|NF|DEFLATE|DDP5\.1|720p|1080p|Complete|Original|S\d{2}|E\d{2})', '', title, flags=re.IGNORECASE)
    title = title.replace('.', ' ')
    title = re.sub(r'\s+', ' ', title).strip()
    return title

def search_episode_by_title(tv_id, season_number, episode_title):
    season = Season()
    season = season.details(tv_id, season_number)
    
    # Search for the episode by title
    for episode in season.episodes:
        if episode.name.lower() in episode_title.lower():
            return episode
    return None

def is_video_file(filename):
    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    return os.path.splitext(filename)[1].lower() in video_extensions

def index_files_by_show(root_path):
    show_index = {}
    season_index = {}
    exclude_file_types = json.loads(os.getenv('IGNORED_FILE_TYPES', '[]'))
    excluded_files = json.loads(os.getenv('IGNORED_FILES', '[]'))
    
    def dfs(path, show_name, show_clean, flag):
        try:
            contents = os.listdir(path)
            for item in contents:
                full_path = os.path.join(path, item)
                if os.path.isdir(full_path):
                    flag = True
                    dfs(full_path, show_name, show_clean, flag)
                elif os.path.isfile(full_path) and is_video_file(item):
                    if (os.path.basename(path) == show_name) and flag:
                        flag = False

                    if (os.path.splitext(item)[1] not in exclude_file_types) and (item not in excluded_files):
                        show_index[show_clean].append(full_path)
                        if flag:
                            parsed_info = parse_filename(item)
                            season_index[show_clean][parsed_info.get("season")] = os.path.basename(path)
                            flag = False

        except PermissionError:
            print(f"Permission denied: {path}")
    
    for show in os.listdir(root_path):
        show_path = os.path.join(root_path, show)
        if os.path.isdir(show_path):
            show_clean = clean_tv_show_title(show)
            show_index[show_clean] = []
            season_index[show_clean] = {}
            flag = True
            dfs(show_path, show, show_clean, flag)
    
    return show_index, season_index

season_keys = [
                'season_number', 'show_id', 'episodes', 'air_date',
                'overview', 'poster_path', 'vote_average', 'vote_count',
                'crew', 'guest_stars', 
            ]

def add_show_metadata(metadata, metadata_type):
    json_metadata = json.dumps(metadata)
    match metadata_type:
        case 'show':
            r.set(f'show:{metadata["name"]}', json_metadata)
        case 'season':
            r.set(f'show:{metadata["show_name"]}:season:{metadata["season_number"]}', json_metadata)
        case 'episode':
            r.set(f'show:{metadata["show_name"]}:season:{metadata["season_number"]}:episode:{metadata["episode_number"]}', json_metadata)
        case _:
            print('Invalid metadata type')
            
def fetch_and_add_metadata(show_index, season_index):
    tv = TV()
    season = Season()
    episode = Episode()

    keys_to_keep = [
            'id', 'name', 'genres', 'first_air_date', 'number_of_episodes', 
            'number_of_seasons', 'overview', 'in_production', 'languages', 
            'networks', 'original_language', 'original_name', 'popularity', 
            'poster_path', 'backdrop_path', 'status', 'vote_average', 'vote_count', 
            'episode_run_time', 'homepage', 'production_companies', 
            'production_countries', 'cast', 'crew'
        ]

    season_keys = [
                'season_number', 'show_id', 'episodes', 'air_date',
                'overview', 'poster_path', 'vote_average', 'vote_count',
                'crew', 'guest_stars', 'name'
            ]
    
    episode_keys = [
                'air_date', 'episode_number', 'name', 'overview',
                'production_code', 'season_number', 'show_id', 'still_path',
                'vote_average', 'vote_count', 'crew', 'guest_stars'
            ]

    for show in show_index:
        search_results = tv.search(show)
        results = list(search_results.results)
        if not(results):
            cleaned_title = clean_tv_show_title(show)
            search_results = tv.search(cleaned_title)
            results = list(search_results.results)

        distances_with_items = [(editDistance(show.lower(), x.name.lower()), x) for x in results]
        min_distance = min(distances_with_items, key=lambda x: x[0])[0]
        closestMatches = [item for distance, item in distances_with_items if distance == min_distance]
        closestMatch = max(closestMatches, key=lambda x: x.popularity)
        if min_distance > 3:
            yesNo = input(f"Confirm that this is the correct show for folder {show}: {closestMatch.name} [y/n]: ")
            if yesNo.lower() == 'n':
                continue

        show_id = closestMatch.id
        show_metadata = tv.details(show_id)
        show_metadata = show_metadata._json
        show_metadata = {key: show_metadata.get(key) for key in keys_to_keep}
        show_metadata["path"] = os.path.join(os.getenv("MEDIA_PATH"), show)
        add_show_metadata(show_metadata, 'show')

        file_paths = show_index[show]
        # parsed_filenames = parse_filenames(file_paths)
        for season_num in season_index[show]:
            season_metadata = season.details(show_id, season_num)
            season_metadata = season_metadata._json
            season_metadata = {key: season_metadata.get(key) for key in season_keys}
            season_metadata["path"] = None
            if season_index[show] != {}:
                season_metadata["path"] = os.path.join(show_metadata["path"], season_index[show][season_num])
            season_metadata["show_name"] = show_metadata["name"]
            add_show_metadata(season_metadata, 'season')

        for file_path in file_paths:
            file = os.path.basename(file_path)
            file = parse_filename(file)
            
            if file:
                seasonNum = file.get('season')
                episodeNum = file.get('episode')
            else:
                return

            if episodeNum == "unknown":
                episode_title = os.path.splitext(os.path.basename(file_path))[0]
                episode_title = re.sub(r'[.\-_]', ' ', episode_title)
                episode_title = clean_tv_show_title(episode_title)
                print(f"Searching by title: {file_path}, {season_metadata["name"]}, {episode_title}")
                episode_metadata = search_episode_by_title(show_id, seasonNum, episode_title)
                if not episode_metadata:
                    print(episode_title)
            else:
                try:
                    episode_metadata = episode.details(show_id, seasonNum, episodeNum)
                except:
                    print(f"Episode not found: {file_path}, {seasonNum}, {episodeNum}")
                    continue

            episode_metadata = episode_metadata._json
            episode_metadata = {key: episode_metadata.get(key) for key in episode_keys}
            episode_metadata["path"] = file_path
            episode_metadata["show_name"] = show_metadata["name"]
            add_show_metadata(episode_metadata, 'episode')
    
    return None


In [ ]:
r.flushdb()
file_path = os.getenv('MEDIA_PATH')
show_index, season_index = index_files_by_show(file_path)
fetch_and_add_metadata(show_index, season_index)
# with open("burner.txt", "w") as f:
#   print(season_index, file=f)

In [ ]:
api_key = os.getenv('TMDB_API_KEY')
tmdb = TMDb()
tmdb.api_key = api_key
show_title = 'Cowboy Bebop'
seasonNum = 1
episodeNum = 1
tv = TV()

results = tv.search(show_title)
print(results)

# metadata = fetch_metadata(show_title, seasonNum, episodeNum)
# print(metadata)

In [ ]:
# for title in result[1]:
#     clean_title = re.sub(r'\.S\d{2}.*', '', title).replace('.', ' ')
#     print(clean_title)
#     print()